In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                        ### LIBRERIAS ###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Importa la librearia pandas para manejo de los datos
import pandas as pd
# Importa la libreria re para indicar expresiones regulares
import re

# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                        ### VARIABLES ###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Ruta del dataset previamente transformado en el script de primera transformacion
path_data = "../Data/movies_transformado.csv"
# aolo columnas a utilizar, por lo que serán integradas en el dataframe
columns_to_use = ['belongs_to_collection','budget','genres','id','original_language',
                    'overview','popularity','production_companies','production_countries',
                    'release_date','revenue','runtime','spoken_languages','status',
                    'title','vote_average','vote_count']
# Lista de columnas numéricas que tienen datos nulos y serán llenados con cero 
columns = ['revenue', 'budget']
# Carga del dataset que contiene la informacion a tranformar, sólo se ocupan ciertas columnas 
df_orig = pd.read_csv(path_data, usecols=columns_to_use, encoding='utf-8', sep=',')
# Se realiza una copia del DataFrame para trabajar en el 
df_transformed = df_orig.copy()
# Variable sobre la que trabaja la desanidación, extrae el nombre o id, para id, presenta problemas
extract = 'name' #'name' or 'id'
# Lista de columnas a separar o desanidar
columns_to_break = ['belongs_to_collection','genres','production_companies','production_countries','spoken_languages']
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# ruta del dataset de creditos 
path_data_credits = "../Data/credits.csv"
# Variable del dataframe
df_credits = pd.read_csv(path_data_credits, encoding='utf-8')
# Se crea la copia del dataframe
df_credits_reg = df_credits.copy()
# Columnas a desanidar
columns_to_break_credits = ['cast', 'crew']


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                                ### FUNCIONES DESANIDACIÓN ###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Función que calcula retorno de inversión.
def column_return(row):
  # Si el valor en la columna "budget" es igual a cero, se devuelve 0 como retorno de inversión.
  if row['budget'] == 0:
    return 0
  else:
    # Se calcula el retorno de inversión dividiendo los valores entre las columnas "revenue" y "budget".
    return row['revenue'] / row['budget']

def extract_actor_names_regex(string):
    # Buscar todas las ocurrencias de name y las extrae
    pattern = r"'name': '([^']+)'"      # Patron que se buscara en la cadena de texto
    actors = re.findall(pattern, string)
    return actors

def extract_director_names_regex(string):
    # Buscar todas las ocurrencias de 'job': 'Director' y extraer el nombre correspondiente
    pattern = r"'job': 'Director'.*?'name': '([^']+)'"      # Patron que se buscara en la cadena de texto
    directors = re.findall(pattern, string)
    return directors

# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                        ### TRANSFORMACIONES DATASET ()###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Ciclo que itera sobre columnas numéricas y llena nulos con ceros, además convierte a tipo int64
for elem in columns:
    df_transformed[elem] = df_transformed[elem].fillna(0)
    # cambia el tipo de dato de la columna budget y  de str a float
    df_transformed[elem] = df_transformed[elem].astype('int64')

#Elimina filas con datos nulos en la columna 'release_date'
df_transformed.dropna(subset=['release_date'], inplace=True)

# Convierte la columna release_date al formato específicado, tipo DateTime 
df_transformed['release_date'] = pd.to_datetime(df_transformed['release_date'], format = '%Y-%m-%d')

# Crea la columna release_year que contiene sólo el año de la fecha de estreno
df_transformed['release_year'] = df_transformed['release_date'].dt.year

# Crea la columna 'return' con valores iguales a cero, se divide 'revenue entre budget' en caso de que budget sea cero,
# el valor será cero
df_transformed['return'] = 0
df_transformed['return'] = df_transformed.apply(column_return, axis = 1)
# Se reemplazan los valores faltantes con 0
df_transformed['return'] = df_transformed['return'].fillna(0)
# Transforma la columna a tipo float de 2 decimales
df_transformed['return'] = pd.Series([round(val, 2) for val in df_transformed['return']])
#rellena los valores nulos de la columna return con ceros
df_transformed['return'] = df_transformed['return'].fillna(0)



# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                        ### EJECUCIONES DE FUNCIONES ###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Ciclo que itera sobre las columnas a desanidar
for colum in columns_to_break:
    df_transformed[colum] = df_transformed[colum].fillna('[]')  # 
    df_transformed[colum] = df_transformed[colum].apply(lambda x: extract_actor_names_regex(x))


df_credits_reg['director'] = df_credits_reg['crew'].apply(lambda x: extract_director_names_regex(x))
df_credits_reg['actor_names'] = df_credits_reg['cast'].apply(lambda x: extract_actor_names_regex(x))

# Elimina las columnas que les extrajimos los datos
df_credits_reg = df_credits_reg.drop(columns=columns_to_break_credits)


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                ### UNION DE AMBOS DATAFRAMES POR ID
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Une los dataframes que fueron generados al limpiar los datos 
df_transformed = df_transformed.merge(df_credits_reg, on='id', how='inner')


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                ### TRANSFORMACIONES ARRAY TO TXT 
# -------------------------------------------------------------------------------------------------------------------------------------------------- #


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                ### EXPORTAR DF EN FORMATO PARQUET 
# -------------------------------------------------------------------------------------------------------------------------------------------------- #

df_transformed.to_parquet('../Data/dataset_movies.parquet', compression='brotli')

---
---
---
Lo siguiente, son pruebas de código, no es necesario ejecutar 
---
---

In [2]:
df_transformed['return'] == 0

0        False
1        False
2         True
3        False
4         True
         ...  
45449     True
45450     True
45451     True
45452     True
45453     True
Name: return, Length: 45454, dtype: bool

In [3]:
df_transformed['return']

0        12.45
1         4.04
2         0.00
3         5.09
4         0.00
         ...  
45449     0.00
45450     0.00
45451     0.00
45452     0.00
45453     0.00
Name: return, Length: 45454, dtype: float64

In [4]:
df = df_transformed.copy()

In [8]:
# Función de apoyo para buscar actores en las columas
def buscar_director(nombre_director):
    """
    Esta función corrobora si el nombre de un director recibido por parámetro se encuentra en el dataset,
    crea una columna llamada count por pelucila si aparece el director en ella
    filtra las peliculas, con la fecha de lanzamiento 
    Parametros: 
        nombre_director: el nombre del  director a buscar
    Retorno: 
        total_count: suma de las veces que aprecio el actor en todos los datos
    """
    nombre_director = nombre_director.lower()
    # Contar el número de veces que aparece el actor en cada fila
    df['count'] = df['director'].apply(lambda directors: sum(director.lower() == nombre_director for director in directors))
    # variable que guarda la suma de todos los conteos
    total_count = df['count'].sum()
    
    # Mascara que busca y filtra la columna count cuando es diferente de cero, es decir, aparece el director
    filtro = df[df['count'] != 0]
    # variable que contiene las peliculas en las que ha participado el actor con año de lnazamiento, retorno, ganancia y retorno
    # Mascara que busca y filtra las columnas cuando count es diferente de cero, es decir, aparece el director
    films = filtro[['title','release_year','return','budget', 'revenue']]
    # Variable que contiene la suma total del retorno del autor 
    retorno = filtro['return'].sum()
    # Calculo del promedio de retorno por película
    promedio = retorno / films.count()
    # Variables a retornar 
    return total_count, films, retorno, round(promedio,2), nombre_director

In [9]:
total_count, films, retorno, promedio, nombre = buscar_director('Steven Spielberg')

In [11]:
def listar_peliculas(films):
    for i in range(len(films)):
        movie = {
            'titulo': str(films.iloc[i]['title']),
            'año': str(films.iloc[i]['release_year']),
            'retorno_pelicula': str(films.iloc[i]['return']),
            'ganancia': str(films.iloc[i]['revenue']),
            'presupuesto': str(films.iloc[i]['budget'])
        }

    return(i, movie)

In [ ]:
i, movie = listar_peliculas(films)

In [ ]:
lista_pelis = []
for i,film in films.iterrows():
    pelicula = {
        'titulo': film['title'],
        'año': film['release_year'],
        'retorno_pelicula': film['return'],
        'ganancia': film['revenue'],
        'presupuesto': film['budget']
    }
    lista_pelis.append(pelicula)

print({
        'name': nombre,
        'catidad_peliculas': total_count,
        'retorno_total': retorno,
        'peliculas': lista_pelis
        })

{'name': 'steven spielberg', 'catidad_peliculas': np.int64(33), 'retorno_total': np.float64(76.04999999999998), 'peliculas': [{'titulo': 'Jurassic Park', 'año': 1993, 'retorno_pelicula': 14.6, 'ganancia': 920100000, 'presupuesto': 63000000}, {'titulo': "Schindler's List", 'año': 1993, 'retorno_pelicula': 14.61, 'ganancia': 321365567, 'presupuesto': 22000000}, {'titulo': 'E.T. the Extra-Terrestrial', 'año': 1982, 'retorno_pelicula': 0.0, 'ganancia': 792965326, 'presupuesto': 10500000}, {'titulo': 'Raiders of the Lost Ark', 'año': 1981, 'retorno_pelicula': 9.91, 'ganancia': 389925971, 'presupuesto': 18000000}, {'titulo': 'Indiana Jones and the Last Crusade', 'año': 1989, 'retorno_pelicula': 3.53, 'ganancia': 474171806, 'presupuesto': 48000000}, {'titulo': 'Jaws', 'año': 1975, 'retorno_pelicula': 4.29, 'ganancia': 470654000, 'presupuesto': 7000000}, {'titulo': 'The Lost World: Jurassic Park', 'año': 1997, 'retorno_pelicula': 0.0, 'ganancia': 229074524, 'presupuesto': 73000000}, {'titulo':

In [ ]:
missing_values_count = df_transformed.isnull().sum()
missing_values_count

belongs_to_collection      0
budget                     0
genres                     0
id                         0
original_language         11
overview                 941
popularity                 0
production_companies       0
production_countries       0
release_date               0
revenue                    0
runtime                  246
spoken_languages           0
status                    80
title                      0
vote_average               0
vote_count                 0
release_year               0
return                     0
director                   0
actor_names                0
dtype: int64

In [ ]:
df_transformed

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,title,vote_average,vote_count,release_year,return,director,actor_names
0,[Toy Story Collection],30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,81.0,[English],Released,Toy Story,7.7,5415,1995,12.45,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,[],65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,104.0,"[English, Français]",Released,Jumanji,6.9,2413,1995,4.04,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,[Grumpy Old Men Collection],0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,101.0,[English],Released,Grumpier Old Men,6.5,92,1995,0.00,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,[],16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,127.0,[English],Released,Waiting to Exhale,6.1,34,1995,5.09,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,[Father of the Bride Collection],0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,106.0,[English],Released,Father of the Bride Part II,5.7,173,1995,0.00,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45449,[],0,"[Drama, Action, Romance]",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[Westdeutscher Rundfunk (WDR), Working Title F...","[Canada, Germany, United Kingdom, United State...",1991-05-13,...,104.0,[English],Released,Robin Hood,5.7,26,1991,0.00,[John Irvin],"[Patrick Bergin, Uma Thurman, David Morrissey,..."
45450,[],0,[Drama],111109,tl,An artist struggles to finish his work while a...,0.178241,[Sine Olivia],[Philippines],2011-11-17,...,360.0,[],Released,Century of Birthing,9.0,3,2011,0.00,[Lav Diaz],"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45451,[],0,"[Action, Drama, Thriller]",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],[United States of America],2003-08-01,...,90.0,[English],Released,Betrayal,3.8,6,2003,0.00,[Mark L. Lester],"[Erika Eleniak, Adam Baldwin, Julie du Page, J..."
45452,[],0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,[Yermoliev],[Russia],1917-10-21,...,87.0,[],Released,Satan Triumphant,0.0,0,1917,0.00,[Yakov Protazanov],"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


In [ ]:
director = 'Tom Hanks'
count, films, retorno, promedio = buscar_director(director)
if not films.empty:
        # Se crea una lista vacia que contendrá un diccionario
        list_films = []
        #
        for i,film in films.iterrows():
            # print(film['title'])
            movie = {
                'titulo': film['title'],
                'anio': film['release_year'],
                'retorno_pelicula': film['return'],
                'ganancia': film['revenue'],
                'presupuesto': film['budget']
        }
            list_films.append(movie)
        print ({
                'director': director,
                'cantidad_peliculas': count,
                'retorno_total_director': retorno,
                'peliculas': list_films
                })

{'director': 'Tom Hanks', 'cantidad_peliculas': np.int64(3), 'retorno_total_director': np.float64(0.12), 'peliculas': [{'titulo': 'That Thing You Do!', 'anio': 1996, 'retorno_pelicula': 0.12, 'ganancia': 34585416, 'presupuesto': 0}, {'titulo': 'Larry Crowne', 'anio': 2011, 'retorno_pelicula': 0.0, 'ganancia': 36160375, 'presupuesto': 30000000}, {'titulo': 'Band of Brothers', 'anio': 2001, 'retorno_pelicula': 0.0, 'ganancia': 0, 'presupuesto': 125000000}]}


In [ ]:
df_transformed['actor_names'].head(5)

0    [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1    [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2    [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
3    [Whitney Houston, Angela Bassett, Loretta Devi...
4    [Steve Martin, Diane Keaton, Martin Short, Kim...
Name: actor_names, dtype: object

In [ ]:
df = df_transformed.copy()

In [ ]:
def buscar_actor(nombre_actor):
    nombre_actor = nombre_actor.lower()
    # Contar el número de veces que aparece el actor en cada fila
    df['count'] = df['actor_names'].apply(lambda actors: sum(actor.lower() == nombre_actor for actor in actors))
    # Sumar todos los conteos
    total_count = df['count'].sum()
    # Mascara que busca y filtra la columna count cuando es diferente de cero, es decir, aparece el actor
    filtro = df[df['count'] != 0]
    # variable que contiene las peliculas en las que ha participado el actor
    films = filtro['title']
    # Variable que contiene la suma total del retorno del autor 
    retorno = filtro['return'].sum()
    # Calculo del promedio de retorno por película
    promedio = retorno / films.count()
    # Variables a retornar 
    return nombre_actor.capitalize(), total_count, films, retorno, round(promedio,2)




In [ ]:
# Ejemplo de uso
actor_name, count, films, retorno, promedio = buscar_actor('Jim ney')
print(f"El actor {actor_name} ha aparecido {count} veces.{retorno}, {promedio} \n {films}")

El actor Jim ney ha aparecido 0 veces.0.0, nan 
 Series([], Name: title, dtype: object)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_25836\1211015576.py:14: RuntimeWarning: invalid value encountered in scalar divide
  promedio = retorno / films.count()


In [ ]:
films

Series([], Name: title, dtype: object)